### 노트북 설명

이 노트북은 사용자 지정 전략이 시스템에서 올바르게 작동하는지 테스트하고 확인하기 위한 파일입니다. Google Colab 환경에 맞게 설정되어 있으며, 로컬 환경에서 사용하는 경우 경로 설정에 유의해야 합니다.

**주요 단계:**

1.  **Google Drive 마운트:** Google Drive에 연결하여 필요한 파일에 접근합니다.
2.  **샘플 데이터 로드:** `/content/drive/MyDrive/NeoMatrixAI/df_sample.csv` 경로에서 샘플 데이터를 읽어 DataFrame으로 로드합니다. 'datetime' 컬럼을 인덱스로 설정합니다.
3.  **선물 전략 검증:**
    *   `drive.MyDrive.NeoMatrixAI.futures.simple_momentum.simple_momentum` 경로의 전략 파일과 `drive.MyDrive.NeoMatrixAI.futures.simple_momentum.simple_momentum_config` 경로의 설정 파일을 가져옵니다.
    *   가져온 전략 및 설정 파일을 사용하여 샘플 데이터를 기반으로 전략을 실행합니다.
    *   전략 실행 결과로 "심볼:비율 수치" 딕셔너리가 출력되는지 확인합니다.

**참고:** 사용자 지정 전략을 테스트하려면 해당 전략 파일과 설정 파일의 가져오기 경로를 수정하고 노트북을 실행하십시오.

#### - Original GitHub Link : https://github.com/NeoMatrixAI/nb-runner/tree/main/notebooks

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Sample Data

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/NeoMatrixAI/df_sample.csv').set_index('datetime')
df

# Verify Futures strategy

In [ ]:
# example strategy, config
import drive.MyDrive.NeoMatrixAI.futures.simple_momentum.simple_momentum as strategy       # Replace with your own strategy file
import drive.MyDrive.NeoMatrixAI.futures.simple_momentum.simple_momentum_config as config  # Replace with your own config file

# run to verify strategy
config_dict = {'strategy_config': config.strategy_config}
weights = strategy.strategy(df, config_dict)
weights